In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for grafic

import os

Метрики качества

In [3]:
def Accuracy(Y_val, Y_pred):
    TP = (Y_val * Y_pred).sum()
    TN = np.logical_not(Y_val | Y_pred).sum()
    return (TP + TN) / len(Y_val)

In [4]:
def Precision(Y_val, Y_pred):
    TP = (Y_val * Y_pred).sum()
    return TP / Y_pred.sum()

In [5]:
def Recall(Y_val, Y_pred):
    TP = (Y_val * Y_pred).sum()
    return TP / Y_val.sum()

In [6]:
def F_metric(Y_val, Y_pred):
    precision = Precision(Y_val, Y_pred)
    recall = Recall(Y_val, Y_pred)
    return 2.0 * recall * precision / (precision + recall)

### Логическая регрессия

Градиентный спуск

In [7]:
def L2_norm(vector):
    return (vector**2).sum()

def L1_norm(vector):
    return np.abs(vec).sum()

In [8]:
def L2_grad(vector):
    return vector

def L1_grad(vector):
    return vector / np.abs(vector)

In [9]:
class GradientDescent:
    def __init__(self, speed, gradient_func, regulasator=None, 
                 C=10.0, eps = 0.001, maxsteps=250):
        self.speed = speed
        self.function = gradient_func
        self.maxsteps = maxsteps
        self.eps = eps
        if regulasator == "l1":
            self.regulasator = lambda w:  L1_grad(w) / C
        elif regulasator == "l2":
            self.regulasator = lambda w: L2_grad(w) / C
        else:
            self.regulasator = lambda w: 0.0
    
    def fit(self, X_train, Y_train):
        # init w0
        w0 = np.zeros(X_train.shape[1])
        w = np.random.random(X_train.shape[1])
        k = 1
        while np.linalg.norm(w - w0) > self.eps and k <= self.maxsteps:
            w0 = w
            temp = self.speed * ((1 / k)**0.5) # like vowpal step temp
            w = w - temp*(self.function(X_train, Y_train, w) + self.regulasator(w))
            k += 1
            
        return w

Логическая регрессия

In [10]:
def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))

def logit_loss(wx, y_real):
    return np.log(1.0 + np.exp(-wx*y_real)).sum()

def logit_grad(x, y, w):
    koeff = (y * sigmoid(-y*x.dot(w)))
    koeff = koeff.reshape((koeff.shape[0], 1)) # make a column
    return -(koeff * x).sum(axis = 0) # full gradient - sum of gradients on ever x[i]


In [11]:
class BinaryLogisticRegression:
    # main params
    def __init__(self, speed = 1.5, reg_type=None, C=2.0, eps=0.001, maxsteps=200):
        # init solver
        self.solver = GradientDescent(speed, logit_grad, reg_type, C, eps, maxsteps)
        # init weight  variable
        self.w = None
        
    # training
    def fit(self, X_train, Y_train):
        # convert 0 to -1 for algo
        Y = np.array(Y_train)
        Y[Y_train == 0] = -1
        # add np.ones colomn for w0 weight:
        x0 = np.ones((X_train.shape[0], 1))
        X = np.hstack((x0, X_train))
        # train weight by gradient descent
        self.w = self.solver.fit(X, Y)
        return self
    
    # returns predictes classes
    def predict(self, X_val, border = 0):
        # add np.ones colomn for w0 weight:
        x0 = np.ones((X_val.shape[0], 1))
        X = np.hstack((x0, X_val))
        # <w, x> product for all examples
        Xw = X.dot(self.w)
        # make predict: 0 - negative, 1 - positive
        Y_pred = np.zeros(Xw.shape).astype(np.int8)
        # a(x) = [<w,x> > t], t - border
        Y_pred[Xw >= border] = 1
        return Y_pred
    
    # probs of positive class
    def predict_proba(self, X_val):
        # add np.ones colomn for w0 weight:
        x0 = np.ones((X_val.shape[0], 1))
        X = np.hstack((x0, X_val))
        # <w, x> product for all examples
        Xw = X.dot(self.w)
        # return proba
        return sigmoid(Xw)
    
    # compute metrics
    def score(self, X_val, Y_val, metric=Accuracy):
        return metric(Y_val, self.predict(X_val))
    
    def weights(self):
        return self.w

    # for fun
    def __str__(self):
        return "Logistic Regression model with gradient descent!"
    
    def __repr__(self):
        return "Logistic Regression"

### Метод опорных векторов

Sequental Minimal Optimization

In [12]:
class Binary_SMO:
    def __init__(self, X, Y, kernel, C, eps, maxsteps, linear):
        self.K = kernel
        self.C = C
        self.tol = eps
        self.maxsteps = maxsteps
        self.X = X
        self.Y = Y
        self.linear = linear
        self.l = np.zeros(Y.shape)
        self.e_cache = -Y.astype(np.float64) # zero prediction - y
        #self.n = len(X)
        self.w0 = 0.0
        if linear:
            self.w = np.zeros(X.shape[1])


    def KKT_violated(self, idx):
        r =  self.Y[idx] * self.e_cache[idx]
        l = self.l[idx]
        # if (M < 1 - tol & l < C) || (M > 1 + tol & l > 0) => KKT violated
        # tol - accuracy of computing
        return (l < self.C and r < -self.tol) or (l > 0.0 and r > self.tol)

    # search by all element ones
    def first_heuristic_one(self):
        for idx in range(self.l.shape[0]):
            if self.KKT_violated(idx):
                yield idx

    # search by all non-bound elements 
    def first_heuristic_two(self):
        # i think here loop faster then pre-choice by numpy
        # because this loop more effective
        for idx in range(self.l.shape[0]):
            if 0.0 < self.l[idx] < self.C:
                if self.KKT_violated(idx):
                    yield idx

    # search elem that maximaze |E1 - E2| from non-boundary
    def second_heuristic_one(self, idx):
        # search |E1 - E2| of each elem
        dE = np.abs(self.e_cache - self.e_cache[idx])
        # all boundary elems not interesting
        dE[(self.l >= self.C) | (self.l <= 0.0)] = 0.0
        # return i2 = argmax|E1 - E2|
        return np.argmax(dE)

    def second_heuristic_two(self, idx):
        mask = (self.l < self.C) & (self.l > 0.0)
        mask[idx] = False
        idxes = np.nonzero(mask)[0]
        order = np.random.permutation(len(idxes))
        return idxes[order]


    def second_heuristic_three(self, idx):
        #if second heuristic without result, get idxs without elems from second
        # it will faster
        mask = (self.l >= self.C) | (self.l <= 0.0)
        mask[idx] = False
        idxes = np.nonzero(mask)[0]
        order = np.random.permutation(len(idxes))
        return idxes[order]

    def get_weights(self):
        if self.linear:
            return self.w

    def get_support(self):
        # returns only support vctors with params if you 
        # dont get them after train
        mask = self.l > 0.0
        return self.X[mask], self.Y[mask], self.l[mask], self.w0

    # return lambdas
    def get_coeffs(self):
        return self.l

    
    def optimize_two(self, i1, i2):
        # it emulates machine e in computations
        eps = 0.00000000000001

        y1 = self.Y[i1]
        y2 = self.Y[i2]
        x1 = self.X[i1]
        x2 = self.X[i2]
        l1 = self.l[i1]
        l2 = self.l[i2]
        E1 = self.e_cache[i1]
        E2 = self.e_cache[i2]

        # compute L H
        if y1 == y2:
            L = max(0.0, l2 + l1 - self.C)
            H = min(self.C, l2 + l1)
        else:
            L = max(0.0, l2 - l1)
            H = min(self.C, self.C + l2 - l1)
        if L == H:
            return False

        # eta
        nu = self.K(x1, x1) + self.K(x2, x2) - 2.0*self.K(x1, x2)

        # compute l2
        if nu > 0.0:
            l2 += y2 * (E1 - E2) / nu
            if l2 < L:
                l2 = L
            elif l2 > H:
                l2 = H
        else:
            c1 = nu/2
            c2 = y2 * (E1 - E2) + nu * l2
            Lobj = c2*L - c1*L*L
            Hobj = c2*H - c1*H*H
            if Lobj > Hobj + eps:
                l2 = L
            elif Lobj < Hobj - eps:
                l2 = H

        if np.abs(l2 - self.l[i2]) < eps*(l2 + self.l[i2] + eps):
            return False

        # compute l1
        l1 -= y1*y2*(l2 - self.l[i2])
        
        if l1 < 0.0:
            l2 += y1*y2*l1
            l1 = 0.0
        elif l1 > self.C:
            l2 += y1*y2*(l1 - self.C)
            l1 = self.C

        # update w0:
        b1 = self.w0 + E1 + y1*(l1 - self.l[i1])*self.K(x1, x1) + y2*(l2 - self.l[i2])*self.K(x1, x2)
        b2 = self.w0 + E2 + y1*(l1 - self.l[i1])*self.K(x1, x2) + y2*(l2 - self.l[i2])*self.K(x2, x2)
        if l1 > 0.0 and l1 < self.C:
            bnew = b1
        elif l2 > 0.0 and l2 < self.C:
            bnew = b2
        else:
            bnew = (b1 + b2) / 2.0
        
        dw0 = bnew - self.w0
        self.w0 = bnew

        # update E_cache
        t1 = y1*(l1 - self.l[i1])
        t2 = y2*(l2 - self.l[i2])
        
        self.e_cache += t1*self.K(self.X, x1) + t2*self.K(self.X, x2) - dw0
        
        #update w:
        if self.linear:
            self.w += t1*x1 + t2*x2
        # update lambdas:
        self.l[i1] = l1
        self.l[i2] = l2

        return True
            
            

    # search 2 lambdas for optimize and change them
    def train(self):
        steps = 0
        non_bound_loop = True
        # main loop
        while steps < self.maxsteps:
            non_bound_loop ^= True
            # outer loops searches i1:
            if not non_bound_loop:
                # h1-1
                changed = False
                for idx1 in self.first_heuristic_one():
                    # h2-1
                    idx2 = self.second_heuristic_one(idx1)
                    if self.optimize_two(idx1, idx2):
                        changed = True
                        continue
                    # h2-2,3 together if h2-1 not worked
                    # concat idxs of heuristics in sequence
                    extra_heuristics = np.concatenate([
                        self.second_heuristic_two(idx1),
                        self.second_heuristic_three(idx1)
                    ])

                    for idx2 in extra_heuristics:
                        if self.optimize_two(idx1, idx2):
                            changed = True
                            break
                steps += 1
                # if nothing changed - work done
                if not changed:
                    break
            else:
                # h1-2
                # while changing itterate non-bound elements
                while changed and steps < self.maxsteps:
                    changed = False
                    # itterate non-bound elements
                    for idx1 in self.first_heuristic_two():
                        # h2-1
                        idx2 = self.second_heuristic_one(idx1)
                        if self.optimize_two(idx1, idx2):
                            changed = True
                            continue
                        # h2-2,3 together if h2-1 not worked
                        # concat idxs of heuristics in sequence
                        extra_heuristics = np.concatenate([
                            self.second_heuristic_two(idx1),
                            self.second_heuristic_three(idx1)
                        ])
                        for idx2 in extra_heuristics:
                            if self.optimize_two(idx1, idx2):
                                changed = True
                                break
                    steps += 1
        # after train return support vectors with lambdas and Y
        mask = self.l > 0.0
        return self.X[mask], self.Y[mask], self.l[mask], self.w0

Метод опорных векторов

In [13]:
def linear_kernel(x1, x2):
    return x1.dot(x2.T)

def rbf_kernel(x1, x2, gamma):
    if len(x1.shape) == 1:
        x1r = x1.reshape((1, x1.shape[0]))
    else:
        x1r = x1
    if len(x2.shape) == 1:
        x2r = x2.reshape((1, x2.shape[0]))
    else:
        x2r = x2
    ans = np.zeros((x1r.shape[0], x2r.shape[0]))
    for i in np.arange(x1r.shape[0]):
        # 
        ans[i] = np.exp(-gamma  * ((x2r - x1r[i])**2).T.sum(axis = 0))
    if len(x1.shape) == 1:
        ans = ans[0]
    if len(x2.shape) == 1:
        ans = ans.T[0]
    return ans

In [14]:
class BinarySVM:
    def __init__(self, kernel = None, C=1.0, eps = 0.001, maxsteps=1000):
        self.C = C
        self.linear = False
        if kernel is None:
            kernel = lambda x1, x2: x1.dot(x2.T)
            self.linear = True
        self.K = kernel
        self.w = None
        self.w0 = None
        self.l = None
        self.svX = None
        self.svY = None
        self.maxsteps = maxsteps
        self.eps = eps
    
    def weights(self):
        if self.linear:
            return np.append(self.w0, self.w)
        
    def predict(self, X_val, border = 0):
        # make predict: 0 - negative, 1 - positive
        Y_pred = np.zeros(X_val.shape[0]).astype(np.int8)
        if self.linear:
            x0 = np.ones((X_val.shape[0], 1))
            X = np.hstack((x0, X_val))
            # <w, x> product for all examples
            Xw = X.dot(np.append(-self.w0, self.w))
            # a(x) = [<w,x> > t], t - border
            Y_pred[Xw >= border] = 1
            return Y_pred
        else:
            yl = (self.svY * self.l).reshape((self.svY.shape[0], 1))
            U = self.K(yl * self.svX, X_val).sum(axis = 0) - self.w0
            Y_pred[U >= border] = 1
            return Y_pred

    def score(self, X_val, Y_val, metric=Accuracy):
        return metric(Y_val, self.predict(X_val))

    def fit(self, X_train, Y_train):
        X = X_train
        Y = np.array(Y_train)
        Y[Y_train == 0] = -1
        smo = Binary_SMO(X, Y, self.K, self.C, self.eps, self.maxsteps, self.linear)
        self.svX, self.svY, self.l, self.w0 = smo.train()
        if self.linear:
            self.w = smo.get_weights()
        return self

    def __str__(self):
        return "Support Vector Machine"

    def __repr__(self):
        return "SVM"
    
    
    # returns support vectors with lambdas
    def vectors(self):
        return self.svX, self.l

### Решающее дерево

In [15]:
class BinaryNode:
    def __init__(self, idxs=None, pos=None, neg=None, c=None):
        self.predicat = None
        self.left = None
        self.right = None
        self.positives = pos
        self.negatives = neg
        self.c = c
        self.idxs = idxs

    # setters:
    def set_left(self, left_node):
        self.left = left_node

    def set_idxs(self, idxs):
        self.idxs = idxs

    def set_right(self, right_node):
        self.right = right_node

    def set_predicat(self, predicat):
        self.predicat = predicat

    def set_class(self, c):
        self.c = c
        
    def set_positives(self, positives):
        self.positives = positives
        
    def set_negatives(self, negatives):
        self.negatives = negatives

    #getters:
    def get_left(self):
        return self.left

    def get_right(self):
        return self.right

    def get_class(self):
        return self.c

    def get_idxs(self):
        return self.idxs
    
    def get_positives(self):
        return self.positives
        
    def get_negatives(self):
        return self.negatives
    
    def get_len(self):
        return self.idxs.shape[0]

    #checkers:
    def is_leaf(self):
        return self.predicat is None

    def is_inner(self):
        return not self.is_leaf()
    
    def make_leaf(self):
        self.predicat = None
        self.left = None
        self.right = None

In [16]:
def bingini(*args):
    # for binary classification: p(-) = 1 - p(+)=>
    # Gini = 2 * p(+) * (1 - p(+))
    if len(args) == 2:
        p = args[0] / (args[1]+args[0])
    else:
        p = args[0].sum() / args[0].shape[0]
    return 2 * p * (1 - p)

def binentropy(*args):
    # for binary classification: p(-) = 1 - p(+)=>
    # Entropy = -p(+)log(p(+)) - (1 - p(+))log(1 - p(+))
    if len(args) == 2:
        p = args[0] / (args[1]+args[0])
    else:
        p = args[0].sum() / args[0].shape[0]
    return -p*np.log(p) - (1 - p)*np.log(p)

In [17]:
class BinaryDescisionTree:
    def __init__(self, criteria=bingini, pruning_cost=None, min_samples_split=2, random_sub_num=None):
        self.CATEGORICAL_LEN = 10
        self.is_categorical = None
        self.categorical_vals = {}
        self.H = criteria
        self.root = None
        self.min_split = min_samples_split
        if random_sub_num is None:
            self.random_subspace = False
        else:
            self.random_subspace = True
            self.feature_num = random_sub_num
        if pruning_cost is None:
            self.pruning = False
        else:
            self.pruning = True
            self.alpha = pruning_cost
            
    def node_classify(self, node):
        # set class num node:
        positives = node.get_positives()
        negatives = node.get_negatives()
        if positives >= negatives:
            node.set_class(1)
        else:
            node.set_class(0)
            
    # create root and recursive building tree
    def build_tree(self):
        # create root with all nums
        self.root = BinaryNode(np.arange(self.Y.shape[0]))
        self.root.set_positives(self.Y.sum())
        negatives = self.root.get_len() - self.root.get_positives()
        self.root.set_negatives(negatives)
        # recursive function of creation
        self.recursive_creation(self.root)
            
            
    # may be modifed
    def stop_criteria(self, node):
        # if num of eelems in node less than min required for split => 1
        if node.get_len() < self.min_split:
            return True
        # if all elems in node has only one class => 1
        positives = node.get_positives()
        negatives = node.get_negatives()
        return (negatives==0 or positives==0)
    
    # union of search_best_split() and split_node()
    def search_best_split(self, node):
        X_iter = self.X[node.get_idxs()]
        Y_iter = self.Y[node.get_idxs()]
        # compute node info criteria
        positiv = node.get_positives()
        negativ = node.get_negatives()
        node_info = self.H(positiv, negativ)
        # best params
        best_gain = 0.0
        best_j, best_t = 0, 0.0
        # search in all features:
        if self.random_subspace:
            # get random permutation
            features = np.random.permutation(self.X.shape[1])
            # stay only feature_num random features
            features = features[:self.feature_num]
        # else search by all features
        else:
            features = range(self.X.shape[1])

        for j in features:
            column = X_iter[:, j]
            # fast search if categorical:
            if self.is_categorical[j]:
                possible_vals = self.categorical_vals[j]
                for i in range(1, possible_vals.shape[0]):
                    mask = column < possible_vals[i]
                    Y_r = Y_iter[mask]
                    if Y_r.shape[0] == 0 or Y_r.shape[0] == Y_iter.shape[0]:
                        continue
                    right_pos = Y_r.sum()
                    right_neg = Y_r.shape[0] - right_pos
                    right_gini = self.H(right_pos, right_neg)
                    left_gini = self.H(positiv - right_pos, negativ - right_neg)
                    # Q(Rm, j, t) = H(Rm) - (|Rl|/|Rm|)H(Rl) - (|Rr|/|Rm|)H(Rr)
                    gain = node_info
                    gain -= (Y_r.shape[0]*right_gini/node.get_len())
                    gain -= (1 - Y_r.shape[0]/node.get_len())*left_gini
                    if gain > best_gain:
                        best_t = possible_vals[i]
                        best_j = j
                        best_gain = gain  
                continue
            # else standart search:
            sorted_col = np.argsort(column)
            right_neg = 0
            right_pos = 0
            last_t = column[sorted_col[0]]
            for i in range(1, column.shape[0]):
                if Y_iter[sorted_col[i-1]]:
                    right_pos += 1
                else:
                    right_neg += 1
                    
                idx = sorted_col[i]
                if column[idx] == last_t:
                    continue
                
                last_t = column[idx]
                # compute gain:
                right_gini = self.H(right_pos, right_neg)
                left_gini = self.H(positiv - right_pos, negativ - right_neg)
                # Q(Rm, j, t) = H(Rm) - (|Rl|/|Rm|)H(Rl) - (|Rr|/|Rm|)H(Rr)
                gain = node_info
                gain -= (i*right_gini/node.get_len()) + (1 - i/node.get_len())*left_gini
                # needs best gain split
                if gain > best_gain:
                    best_t = column[idx]
                    best_j = j
                    best_gain = gain
                    
        if best_gain > 0.0:
            return best_j, best_t
    
    # create 2 new nodes: left and right
    def split_node(self, node, j, t):
        # set predicat rule for node
        predicat = lambda x: x[j] < t
        node.set_predicat(predicat)
        # get split mask
        column = self.X[node.get_idxs(), j]
        mask = column < t
        # make idxs for left and right
        right_idxs = node.get_idxs()[mask]
        left_idxs = node.get_idxs()[np.logical_not(mask)]
        #compute positives:
        right_pos = self.Y[right_idxs].sum()
        left_pos = self.Y[left_idxs].sum()
        # compute negatives
        right_neg = right_idxs.shape[0] - right_pos
        left_neg = left_idxs.shape[0] - left_pos
        # create nodes
        node.set_left(BinaryNode(left_idxs, left_pos, left_neg))
        node.set_right(BinaryNode(right_idxs, right_pos, right_neg))
    
    # recursive function for nodes:
    def recursive_creation(self, node):
        # classify another node:
        self.node_classify(node)
        # stop criteria for building
        if self.stop_criteria(node):
            return
        #else find best split
        jt = self.search_best_split(node)
        # if we cant find best split - stop
        if jt is None:
            return

        # split node for 2 child:
        self.split_node(node, *jt)
        # start recursion for left:
        self.recursive_creation(node.get_left())
        # for right:
        self.recursive_creation(node.get_right())
    
    def tree_pruning(self, node):
        # this node R_a(t) computing:
        # R = sum([y != c]) / |N|
        if node.get_class():
            R_a = node.get_negatives() / node.get_len()
        else:
            R_a = node.get_positives() / node.get_len()
        # R_a(t) = R(t) + a
        R_a += self.alpha
        # at first go while not leaf:
        if node.is_leaf():
            # return R_a(leaf)
            return R_a
        # R_a(Tl) and R_a(Tr)
        R_al = self.tree_pruning(node.get_left())
        R_ar = self.tree_pruning(node.get_right())
        # if R_a(t) < R_a(T) => pruning
        if R_a <= R_al + R_ar:
            node.make_leaf()
            return R_a
        # else do nothing
        return R_al + R_ar
    
    def search_categorical(self):
        self.is_categorical = np.zeros(self.X.shape[1]).astype(np.int8)
        for j in range(self.X.shape[1]):
            uniq = np.unique(self.X[:, j])
            if uniq.shape[0] < self.CATEGORICAL_LEN:
                self.categorical_vals[j] = uniq
                self.is_categorical[j] = 1
            

    def fit(self, X_train, Y_train):
        # temp sets for comfort
        self.X = X_train
        self.Y = Y_train
        # we will store only idxs while training in nodes =>
        # make idxs column for comfort:
        self.idxs = np.arange(X_train.shape[0])
        # for speed search categorical
        self.search_categorical()
        # build tree 
        self.build_tree()
        # pruning tree
        if self.pruning:
            self.tree_pruning(self.root)
        #  delete temp sets:
        #self.free_memory(root)
        del self.X
        del self.Y
        del self.idxs
        return self

    def predict(self, X_val):
        Y_pred = np.zeros(X_val.shape[0]).astype(np.int8)
        # for each elem in X predict result:
        for i in np.arange(X_val.shape[0]):
            Y_pred[i] = self.predict_one(X_val[i])
        return Y_pred

    def predict_one(self, x):
        node = self.root
        while node.is_inner():
            if node.predicat(x):
                node = node.get_right()
            else:
                node = node.get_left()
        # if in leaf:
        return node.get_class()

    def score(self, X_val, Y_val, metric=Accuracy):
        return metric(Y_val, self.predict(X_val))

    def __str__(self):
        return "Decision Tree"

    def __repr__(self):
        return "Tree"

### Метод k-ближайших соседей

In [18]:
def minkovski(x1, x2, p = 3):
    return (np.abs(x1 - x2) ** p).T.sum(axis = 0)**(1.0 / p)

def euclid(x1, x2):
    return minkovski(x1, x2, 2)

In [19]:
def biquadratical_kernel(t):
    return (-t**2 + 1)**2

def triquadratical_kernel(t):
    return (-t**2 + 1)**3
    
def triqubical_kernel(t):
    return (-t**3 + 1)**3

In [20]:
class BinaryKNN:
    def __init__(self, k, metric = euclid, parsen_kernel = None):
        self.k = k
        self.p = metric
        # algo shoud remember all data
        self.X = None
        self.Y = None
        if parsen_kernel is None:
            # all elems have equal weight
            self.Kp = lambda t: 1.0
        else:
            # parsen method
            self.Kp = parsen_kernel

    def stolp_filtration(self):
        pass
    
    def fit(self, X_train, Y_train):
        # just remmember data:
        self.X = X_train
        self.Y = Y_train
        # check correct of k
        self.k = min(self.k, len(self.Y) - 1)
        return self

    def predict(self, X_val):
        Y_pred = np.zeros(len(X_val)).astype(np.int8)
        # for each elem in X predict result:
        for i in np.arange(len(X_val)):
            Y_pred[i] = self.predict_one(X_val[i])
        return Y_pred

    def predict_one(self, x):
        # compute all distances
        r_x = self.p(self.X, x)
        # take sorted order of dists in idxs
        order = np.argsort(r_x)
        # width for parsen is distance to k+1 neiborh:
        h = r_x[order[self.k]]
        # idxs of first k elems neibrh
        order = order[:self.k]
        # take first k Y:
        Y_k = self.Y[order]
        # compute parsen function for all neiborh:
        K = self.Kp(r_x[order] / h)
        
        # compute functional for positives elems
        pos_w = (K * Y_k).sum()
        # compute functional for negatives elems
        neg_w = (K * np.logical_not(Y_k)).sum()
        
        # class with more functional wins
        return int(pos_w > neg_w) # 0 or 1


    def score(self, X_val, Y_val, metric=Accuracy):
        return metric(Y_val, self.predict(X_val))

    def __str__(self):
        return "k Nearest Neighbor"

    def __repr__(self):
        return "KNN"

### Случайный лес

In [21]:
class BinaryRandomForest:
    def __init__(self, Ntrees=20, criteria=bingini, random_sub_num=None):
        self.N = Ntrees
        self.criteria = criteria
        self.sub_space_num = random_sub_num
        # list for trained models
        self.trees = [None for _ in range(self.N)]

    # get random sample for tree training
    def bootstrap_sample(self, X, Y):
        # indexes of X and Y
        indexes = np.arange(len(Y))
        # rundom indexes with repeats
        indexes = np.random.choice(indexes, len(indexes))
        # bootstrap sample
        return X[indexes], Y[indexes]
    
    def fit(self, X_train, Y_train):
        if self.sub_space_num is None:
            self.sub_space_num = int(len(X_train)**(1/2))
        # train N trees with 
        for i in range(self.N):
            # create tree(use our class of tree)
            self.trees[i] = BinaryDescisionTree(
                self.criteria, # user criteria
                None, # trees without prunning
                2, # build while it possible 
                self.sub_space_num) # num of random features in random space method
            # and train tree:
            self.trees[i].fit(*self.bootstrap_sample(X_train, Y_train))
        return self
            

    def predict(self, X_val):
        voices = np.zeros(len(X_val))
        # make vote:
        for tree in self.trees:
            voices += tree.predict(X_val)
        # compute winners:
        Y_pred = voices >= (self.N + 1) // 2
        return Y_pred.astype(np.int8)

    
    def score(self, X_val, Y_val, metric=Accuracy):
        return metric(Y_val, self.predict(X_val))

    def __str__(self):
        return "Random Forest"

    def __repr__(self):
        return "RF"

### Scikit Learn реализации

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

### Применение 

In [23]:
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [24]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [25]:
train_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.head(3)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
0,13.4,22.9,0.6,5.2,9.35,44.0,20.0,24.0,71.0,22.0,...,0,0,0,0,0,0,0,0,0,1
1,7.4,25.1,0.0,5.2,9.35,44.0,4.0,22.0,44.0,25.0,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,5.2,9.35,46.0,19.0,26.0,38.0,30.0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
year_validate = train_df['Year'].to_numpy()
print("Данные с", year_validate.min(), "по", year_validate.max(), "год.")

Данные с 2007 по 2015 год.


Выделим целевую переменную и нормализуем данные.

In [27]:
Y_train = train_df['RainTomorrow'].to_numpy()
Y_test = test_df['RainTomorrow'].to_numpy()

X_train = train_df.drop('RainTomorrow', axis = 1)
X_test = test_df.drop('RainTomorrow', axis = 1)

In [28]:
scale_features_std = MinMaxScaler()

X_train = scale_features_std.fit_transform(X_train.to_numpy()) 
X_test = scale_features_std.transform(X_test.to_numpy())

In [29]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

Кросс валидация

In [30]:
def time_cross_validation(model_from_param, param_generator):
    max_val = 2015
    min_val = 2013
    best_score = 0.0
    best_param = None
    for p in tqdm(param_generator):
        mean_score = 0.0
        for year in range(min_val, max_val + 1):
            mask_train = year_validate < year
            mask_val = year_validate == year
            X_t = X_train[mask_train]
            Y_t = Y_train[mask_train]
            X_v = X_train[mask_val]
            Y_v = Y_train[mask_val]
            model = model_from_param(p)
            model.fit(X_t, Y_t)
            mean_score += model.score(X_v, Y_v)
        mean_score /= max_val - min_val + 1
        if best_score < mean_score:
            best_score = mean_score
            best_param = p
    return best_param, best_score

Логическая регрессия

In [32]:
generate_c = np.arange(0.1, 1.01, 0.1)
modeling = lambda c: BinaryLogisticRegression(reg_type='l2', C=c, maxsteps=1500, speed=0.02)
best_C, best_score = time_cross_validation(modeling, generate_c)
print("Best param is", best_C, "with accuracy:", best_score)

100%|██████████| 10/10 [1:40:52<00:00, 605.27s/it]  

Best param is 0.30000000000000004 with accuracy: 0.8057910510872309


In [33]:
my_log_l2_model = BinaryLogisticRegression(reg_type='l2', C=best_C, maxsteps=1500, speed=0.02)
my_log_l2_model.fit(X_train, Y_train)

Logistic Regression

In [34]:
my_log_l2_model.score(X_test, Y_test)

0.8151228151228151

Логическая регрессия из sklearn

In [35]:
generate_c = np.arange(0.1, 1.01, 0.1)
modeling = lambda c: LogisticRegression(penalty='l2', C=c, max_iter=250)
best_C, best_score = time_cross_validation(modeling, generate_c)
print("Best param is", best_C, "with accuracy:", best_score)

100%|██████████| 10/10 [00:42<00:00,  4.25s/it]

Best param is 0.6 with accuracy: 0.854795589212209


In [36]:
skl_log_l2_model = LogisticRegression(penalty='l2', C=best_C, max_iter=250)
skl_log_l2_model.fit(X_train, Y_train)

LogisticRegression(C=0.6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=250, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
skl_log_l2_model.score(X_test, Y_test)

0.8421498421498421

Решающее дерево

In [31]:
best_a = 0.000019

In [32]:
my_desc_tree_model = BinaryDescisionTree(pruning_cost=best_a)
my_desc_tree_model.fit(X_train, Y_train)

Tree

In [33]:
my_desc_tree_model.score(X_test, Y_test)

0.8183568183568184

Решающее дерево из sklearn

In [63]:
skl_desc_tree_model = DecisionTreeClassifier()
skl_desc_tree_model.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [64]:
skl_desc_tree_model.score(X_test, Y_test)

0.7714252714252714

Метод k-ближайших соседей из sklearn

In [65]:
generate_k = np.arange(1, 65, 15)
modeling = lambda K: KNeighborsClassifier(n_neighbors=K)
best_k, best_score = time_cross_validation(modeling, generate_k)
print("Best param is", best_k, "with accuracy:", best_score)

100%|██████████| 5/5 [2:02:28<00:00, 1469.80s/it]

Best param is 31 with accuracy: 0.813676096672508


In [66]:
skl_knn_model = KNeighborsClassifier(n_neighbors=best_k)
skl_knn_model.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=31, p=2,
           weights='uniform')

In [67]:
skl_knn_model.score(X_test, Y_test)

0.7934087934087934

Моя реализация KNN

In [68]:
my_knn_model = BinaryKNN(k=best_k)
my_knn_model.fit(X_train, Y_train)

KNN

In [69]:
my_knn_model.score(X_test, Y_test)

0.7934087934087934

Метод опорных векторов из sklearn

In [70]:
generate_c = np.arange(0.1, 1.1, 0.1)
modeling = lambda c: LinearSVC(C=c)
best_C, best_score = time_cross_validation(modeling, generate_c)
print("Best param is", best_C, "with accuracy:", best_score)

100%|██████████| 10/10 [02:04<00:00, 12.40s/it]

Best param is 0.6 with accuracy: 0.8544884970944056


In [71]:
skl_svm_model = LinearSVC(C=best_C)
skl_svm_model.fit(X_train, Y_train)

LinearSVC(C=0.6, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [72]:
skl_svm_model.score(X_test, Y_test)

0.8414183414183414